In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
from kaggle.dataset.spooky import SpookyDataset

/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [49]:
from sklearn import preprocessing

from ensembling.spooky.tfidf import tfidf, count_vec
from ensembling.spooky.lr import lr, lr_svd_gridsearch
from ensembling.spooky.nb import mnb, mnb_gridsearch, generate_nb_features
from ensembling.spooky.svm import svd, svm
from ensembling.spooky.tree import xgbst, xgbst_on_vec
from ensembling.spooky.keras_models import simple_net, bilstm_net

from sarvam.nlp import word_vectors
from sarvam.ensembler import Ensembler



In [4]:
# Use the sarvam utils to load the data set and get train/val/test data
dataset = SpookyDataset(train_file_path="../../../data/spooky_author_identification/input/train.csv",
                       test_file_path= "../../../data/spooky_author_identification/input/test.csv")

TEXT_COL = "text"
CATEGORY_COL = "author"



dataset.prepare()
dataframe = dataset.dataframe

train_x = dataframe.get_train_text_data()
train_y = dataframe.get_train_label()

val_x = dataframe.get_val_text_data()
val_y = dataframe.get_val_label()

text_x = dataframe.get_test_text_data()

train_df = dataframe.get_train_data()
test_df = dataframe.get_test_df()

## Prepare the data for modeling ###
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
# train_y = train_df['author'].map(author_mapping_dict)
train_id = train_df['id'].values
test_id = test_df['id'].values


Loading processed files...
Fitting LabelEncoder and LabelBinarizer on processed audio_utils...
Done!
Preparing vocab file...
No <UNK> token found
experiments/data//spooky_dataset/words_vocab.tsv
Size of train data: 2.772MB
Labels and their document counts based on author
EAP    7505
HPL    5353
MWS    5741
Name: author, dtype: int64
Size of validation data: 0.147MB
Labels and their document counts based on author
EAP    395
HPL    282
MWS    303
Name: author, dtype: int64
Size of test data: 1.240MB


### Text to Numeric Features

### TfIdf

In [44]:
train_x_word_tfidf, val_x_word_tfidf = tfidf(train_x, val_x, ngram=(1,3), analyzer="word", lowercase=True)

In [36]:
train_x_char_tfidf, val_x_char_tfidf = tfidf(train_x, val_x, ngram=(4,7), analyzer="char", lowercase=True)

## CountVec 

In [39]:
train_x_word_ctv, val_x_word_ctv = count_vec(train_x, val_x,  analyzer="word", lowercase=True, ngram=(1,3))

In [40]:
train_x_char_ctv, val_x_char_ctv = count_vec(train_x, val_x,  analyzer="char", lowercase=True, ngram=(4,7))

### SVD

In [46]:
train_x_word_tfidf_svd_scl, val_x_word_tfidf_svd_scl =  svd(train_x_word_tfidf, val_x_word_tfidf, 50)

In [ ]:
train_x_char_tfidf_svd_scl, val_x_char_tfidf_svd_scl =  svd(train_x_char_tfidf, val_x_char_tfidf, 50)

In [48]:
train_x_word_ctv_svd_scl, val_x_word_ctv_svd_scl =  svd(train_x_word_ctv, val_x_word_ctv, 50)

In [ ]:
train_x_char_ctv_svd_scl, val_x_char_ctv_svd_scl =  svd(train_x_char_ctv, val_x_char_ctv, 50)

In [54]:
embeddings_index = word_vectors.load_embeddings("/opt/datasets/glove/glove.840B.300d.txt")

2196017it [03:51, 9470.85it/s]

Found 2195884 word vectors.


In [55]:
train_x_glove = word_vectors.data_2_glove_vec(train_x, embeddings_index=embeddings_index)

100%|██████████| 18599/18599 [00:07<00:00, 2560.90it/s]


In [56]:
val_x_glove = word_vectors.data_2_glove_vec(val_x, embeddings_index=embeddings_index)

100%|██████████| 980/980 [00:00<00:00, 2616.56it/s]


In [46]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
train_x_glove_scl = scl.fit_transform(train_x_glove)
val_x_glove_scl = scl.transform(val_x_glove)

train_y_one_hot_encoded = dataframe.get_train_one_hot_label()
val_y_one_hot_encoded = dataframe.get_val_one_hot_label()

Labels and their document counts based on author
EAP    7505
HPL    5353
MWS    5741
Name: author, dtype: int64
Labels and their document counts based on author
EAP    395
HPL    282
MWS    303
Name: author, dtype: int64


### Logistic Regression

In [12]:
lr(train_x_tfidf, train_y, val_x_tfidf, val_y)

logloss: 0.610 


In [14]:
lr(train_x_ctv, train_y, val_x_ctv, val_y)

logloss: 0.516 


In [24]:
lr_svd_gridsearch(train_x_ctv, train_y, val_x_ctv, val_y)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=180 ................
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=-0.826303, total= 1.3min
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=-0.810406, total= 1.3min
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=180, score=-0.775996, total= 2.3min
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=180, score=-0.779073, total= 2.3min
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__pe

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.0min


[CV]  lr__C=0.1, lr__penalty=l2, svd__n_components=120, score=-0.801700, total= 1.7min
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=-0.812132, total= 1.8min
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=-0.795271, total= 1.7min
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l2, svd__n_components=180, score=-0.771930, total= 2.6min
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l2, svd__n_components=180, score=-0.758431, total= 2.6min
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.9min


[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.814725, total= 1.5min
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.797284, total= 1.5min
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV] lr__C=10, lr__penalty=l1, svd__n_components=120 .................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=-0.781171, total= 2.2min
[CV] lr__C=10, lr__penalty=l1, svd__n_components=120 .................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=-0.756178, total= 2.6min
[CV] lr__C=10, lr__penalty=l1, svd__n_components=180 .................
[CV] lr__C=10, lr__penalty=l1, svd__n_components=180 .................
[CV] lr__C=10, lr__penalty=l2, svd__n_components=120 .................
[CV] lr__C=10, lr__penalty=l2, svd__n_components=120 .................
[CV]  lr__C=10, lr__penalty=l1, svd__n_components=120, score=-0.808256, total= 2.5mi

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/ho

### Naive Bayes

In [25]:
mnb(train_x_tfidf, train_y, val_x_tfidf, val_y)

logloss: 0.567 


In [29]:
mnb(train_x_ctv, train_y, val_x_ctv, val_y)

logloss: 0.449 


In [26]:
mnb_gridsearch(train_x_tfidf, train_y, val_x_tfidf, val_y)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] nb__alpha=0.01 ..................................................
[CV] nb__alpha=0.001 .................................................
[CV] nb__alpha=0.001 .................................................
[CV] nb__alpha=0.01 ..................................................
[CV] ................. nb__alpha=0.001, score=-0.625190, total=   0.1s
[CV] ................. nb__alpha=0.001, score=-0.597814, total=   0.1s
[CV] .................. nb__alpha=0.01, score=-0.515710, total=   0.1s
[CV] nb__alpha=0.1 ...................................................
[CV] .................. nb__alpha=0.01, score=-0.496752, total=   0.1s
[CV] nb__alpha=1 .....................................................
[CV] nb__alpha=0.1 ...................................................
[CV] nb__alpha=1 .....................................................
[CV] ................... nb__alpha=0.1, score=-0.492501, total=   0.0s
[CV] ............

[Parallel(n_jobs=-1)]: Batch computation too fast (0.1818s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.4s finished


Best score: -0.487
Best parameters set:
	nb__alpha: 0.1


In [27]:
mnb_gridsearch(train_x_ctv, train_y, val_x_ctv, val_y)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] nb__alpha=0.001 .................................................
[CV] nb__alpha=0.001 .................................................
[CV] nb__alpha=0.01 ..................................................
[CV] nb__alpha=0.01 ..................................................
[CV] ................. nb__alpha=0.001, score=-1.424465, total=   0.3s
[CV] ................. nb__alpha=0.001, score=-1.486892, total=   0.3s
[CV] nb__alpha=0.1 ...................................................
[CV] nb__alpha=0.1 ...................................................
[CV] .................. nb__alpha=0.01, score=-1.119090, total=   0.3s
[CV] .................. nb__alpha=0.01, score=-1.073956, total=   0.4s
[CV] nb__alpha=1 .....................................................
[CV] nb__alpha=1 .....................................................
[CV] ................... nb__alpha=0.1, score=-0.730633, total=   0.2s
[CV] nb__alpha=10

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.8s remaining:    0.6s


[CV] nb__alpha=100 ...................................................
[CV] .................... nb__alpha=10, score=-0.569758, total=   0.2s
[CV] ................... nb__alpha=100, score=-0.853411, total=   0.1s
[CV] ................... nb__alpha=100, score=-0.853695, total=   0.2s
[CV] .................... nb__alpha=10, score=-0.567352, total=   0.3s


[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.8s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.0s finished


Best score: -0.504
Best parameters set:
	nb__alpha: 1


In [54]:
# generate_nb_features
train_x_word_tfidf_nb_features, val_x_word_tfidf_nb_features =  generate_nb_features(train_x_word_tfidf, train_y, val_x_word_tfidf, 3)

In [56]:
val_x_word_tfidf_nb_features.shape

(980, 3)

### SVM

In [31]:
svm(train_x_svd_scl, train_y, val_x_svd_scl, val_y)

logloss: 0.705 


### XGBoost

In [15]:
xgbst_on_vec(train_x_tfidf, train_y, val_x_tfidf, val_y)

logloss: 0.790 


In [16]:
xgbst_on_vec(train_x_ctv, train_y, val_x_ctv, val_y)

logloss: 0.784 


In [21]:
xgbst(train_x_svd_scl, train_y, val_x_svd_scl, val_y)

logloss: 0.751 


In [31]:
!ls /opt/datasets/glove/

glove.840B.300d.txt	       GoogleNews-vectors-negative300.bin
glove.840B.300d.txt.cache.npy  GoogleNews-vectors-negative300.bin.gz
glove.840B.300d.zip


In [31]:
xgbst(train_x_glove, train_y, val_x_glove, val_y)

logloss: 0.659 


### Keras

In [49]:
simple_net(train_x_glove_scl, train_y_one_hot_encoded, val_x_glove_scl, val_y_one_hot_encoded)

Train on 18599 samples, validate on 980 samples
Epoch 1/5
18599/18599 [==============================] - 16s - loss: 0.8964 - val_loss: 0.6689
Epoch 2/5
18599/18599 [==============================] - 2s - loss: 0.6801 - val_loss: 0.6440
Epoch 3/5
18599/18599 [==============================] - 2s - loss: 0.6185 - val_loss: 0.6250
Epoch 4/5
18599/18599 [==============================] - 2s - loss: 0.5792 - val_loss: 0.6069
Epoch 5/5
18599/18599 [==============================] - 1s - loss: 0.5349 - val_loss: 0.6276


In [64]:
bilstm_net(train_x, train_y_one_hot_encoded,
               val_x, val_y_one_hot_encoded, embeddings_index)

100%|██████████| 25943/25943 [00:00<00:00, 351968.52it/s]


Train on 18599 samples, validate on 980 samples
Epoch 1/100
18599/18599 [==============================] - 153s - loss: 0.9719 - val_loss: 0.8440
Epoch 2/100
18599/18599 [==============================] - 165s - loss: 0.8520 - val_loss: 0.7198
Epoch 3/100
18599/18599 [==============================] - 168s - loss: 0.7864 - val_loss: 0.6656
Epoch 4/100
18599/18599 [==============================] - 150s - loss: 0.7510 - val_loss: 0.6594
Epoch 5/100
18599/18599 [==============================] - 149s - loss: 0.7136 - val_loss: 0.6283
Epoch 6/100
18599/18599 [==============================] - 149s - loss: 0.6805 - val_loss: 0.5926
Epoch 7/100
18599/18599 [==============================] - 149s - loss: 0.6427 - val_loss: 0.5881
Epoch 8/100
18599/18599 [==============================] - 149s - loss: 0.6104 - val_loss: 0.5591
Epoch 9/100
18599/18599 [==============================] - 151s - loss: 0.5823 - val_loss: 0.5587
Epoch 10/100
18599/18599 [==============================] - 149s - los

### Ensembler

In [38]:
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from tc_utils.losses import multiclass_logloss



In [41]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [train_x_tfidf, train_x_ctv, train_x_tfidf, train_x_ctv], 1: [train_x_glove]}
test_data_dict = {0: [val_x_tfidf, val_x_ctv, val_x_tfidf, val_x_ctv], 1: [val_x_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, train_y, lentrain=train_x_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=val_x_glove.shape[0])

[08:56:57] INFO Found 3 classes
[08:56:57] INFO Training Level 0 Fold # 1. Model # 0
[08:56:57] INFO Predicting Level 0. Fold # 1. Model # 0
[08:56:57] INFO Level 0. Fold # 1. Model # 0. Validation Score = 0.675661
[08:56:57] INFO Training Level 0 Fold # 2. Model # 0
[08:56:58] INFO Predicting Level 0. Fold # 2. Model # 0
[08:56:58] INFO Level 0. Fold # 2. Model # 0. Validation Score = 0.660172
[08:56:58] INFO Training Level 0 Fold # 3. Model # 0
[08:56:58] INFO Predicting Level 0. Fold # 3. Model # 0
[08:56:58] INFO Level 0. Fold # 3. Model # 0. Validation Score = 0.668261
[08:56:58] INFO Level 0. Model # 0. Mean Score = 0.668031. Std Dev = 0.006325
[08:56:58] INFO Training Level 0 Fold # 1. Model # 1
[08:57:01] INFO Predicting Level 0. Fold # 1. Model # 1
[08:57:01] INFO Level 0. Fold # 1. Model # 1. Validation Score = 0.577250
[08:57:01] INFO Training Level 0 Fold # 2. Model # 1
[08:57:03] INFO Predicting Level 0. Fold # 2. Model # 1
[08:57:03] INFO Level 0. Fold # 2. Model # 1. Val

In [43]:
# check error:
multiclass_logloss(val_y, preds[1])

0.39777932094927465

### Neural Network Feature Extraction

In [4]:
from sarvam.eda.meta_features.cnn_features import get_cnn_feats

In [5]:
train_pred, test_pred, best_val_train_pred, best_val_test_pred = \
get_cnn_feats(train_df, test_df, TEXT_COL, CATEGORY_COL, rnd=42)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 10)           300000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 16)            496       
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 300,819
Trainable params: 300,819
Non-trainable params: 0
_________________________________________________________________
Trai

Train on 14096 samples, validate on 1567 samples
Epoch 1/10
Epoch 00000: val_loss improved from inf to 0.79902, saving model to /tmp/nn_model.h5
4s - loss: 1.0263 - acc: 0.4780 - val_loss: 0.7990 - val_acc: 0.6886
Epoch 2/10
Epoch 00001: val_loss improved from 0.79902 to 0.47703, saving model to /tmp/nn_model.h5
4s - loss: 0.5569 - acc: 0.7889 - val_loss: 0.4770 - val_acc: 0.8226
Epoch 3/10
Epoch 00002: val_loss improved from 0.47703 to 0.40915, saving model to /tmp/nn_model.h5
3s - loss: 0.3180 - acc: 0.8875 - val_loss: 0.4092 - val_acc: 0.8443
Epoch 4/10
Epoch 00003: val_loss improved from 0.40915 to 0.40559, saving model to /tmp/nn_model.h5
4s - loss: 0.2121 - acc: 0.9276 - val_loss: 0.4056 - val_acc: 0.8481
Epoch 5/10
Epoch 00004: val_loss did not improve
4s - loss: 0.1505 - acc: 0.9508 - val_loss: 0.4279 - val_acc: 0.8513
Epoch 6/10
Epoch 00005: val_loss did not improve
4s - loss: 0.1081 - acc: 0.9667 - val_loss: 0.4618 - val_acc: 0.8468
Epoch 7/10
Epoch 00006: val_loss did not im

In [9]:
train_pred.shape, test_pred.shape, best_val_train_pred.shape, best_val_test_pred.shape

((19579, 3), (8392, 3), (19579, 3), (8392, 3))

## Extra Features

In [10]:
from ensembling.spooky.meta_features.extra_features import *
from ensembling.spooky.meta_features.spacy_features import *

In [ ]:
new_train_hand_features_df = extract_features(train_df, train_flag=True)

 35%|███▍      | 9/26 [03:52<07:18, 25.79s/it]

In [ ]:
new_test_hand_features_df = extract_features(test_df, train_flag=True)

In [22]:
train_spacy_features = get_spacy_features(train_df, TEXT_COL)

In [23]:
test_spacy_features = get_spacy_features(test_df, TEXT_COL)

100%|██████████| 8392/8392 [03:08<00:00, 44.46it/s]

done creating spacy features 188.77141976356506


In [20]:
train_spacy_features

,pos_txt,tag_txt,dep_txt
0,DET NOUN PUNCT ADV PUNCT VERB PRON DET NOUN A...,"DT NN , RB , VBD PRP DT NNS IN VBG DT NNS IN ...",det nsubj punct advmod punct ccomp dobj det d...
1,PRON ADV ADV VERB ADP PRON ADP DET NOUN VERB ...,PRP RB RB VBD IN PRP IN DT NN MD VB DT JJ NN .,nsubj neg advmod ROOT prep pobj mark det nsub...
2,ADP ADJ ADJ NOUN VERB DET ADJ NOUN NOUN PUNCT...,"IN PRP$ JJ NN VBD DT JJ NN NN , IN WDT , IN P...",prep poss amod pobj ROOT det compound compoun...
3,ADV ADJ VERB NOUN ADP PRON VERB ADP PROPN PRO...,WRB JJ VBZ NN IN PRP VBD IN NNP NNP IN DT CD ...,advmod acomp ROOT nsubj mark nsubj advcl prep...
4,VERB NOUN ADV PUNCT ADV ADV NOUN PUNCT DET PR...,"VBG NN RB , RB RB NN , DT NNP VBD PRP$ NNS : ...",advcl dobj advmod punct neg advmod dep punct ...
5,DET NOUN VERB ADP NOUN PUNCT ADJ ADJ NOUN VER...,"DT NN VBD IN NN , PRP$ JJS NNS VBN IN PRP$ JJ...",det nsubj acl prep pobj punct poss amod nsubj...
6,DET NOUN PUNCT ADV PUNCT ADP DET NOUN PUNCT V...,"DT NN , RB , IN DT NN , VBD NN IN DT NN IN AF...",det nsubj punct advmod punct prep det pobj pu...
7,DET NOUN VERB ADP NOUN ADP ADJ NOUN PUNCT,DT NN VBD IN NNS IN PRP$ NN .,det nsubj ROOT prep pobj prep poss pobj punct
8,PRON VERB ADP PRON VERB ADV VERB ADP PRON PUN...,PRP VBD IN PRP MD RB VB IN PRP `` NN '' IN VB...,nsubj ROOT mark nsubj aux neg ccomp prep poss...
9,PRON VERB ADP CCONJ DET NOUN ADP NOUN PUNCT C...,"PRP VBP IN CC DT NN IN NNS , CC DT NN IN NNS ...",nsubj ROOT mark preconj det nsubj prep pobj p...


In [26]:
train_x_spacy_tag_cnt_feature, test_x_spacy_tag_cnt_feature = count_vec(train_spacy_features["tag_txt"].values, test_spacy_features["tag_txt"].values, lowercase=False, ngram=1)

In [29]:
train_x_spacy_pos_cnt_feature, test_x_spacy_pos_cnt_feature = count_vec(train_spacy_features["pos_txt"].values, test_spacy_features["pos_txt"].values, lowercase=False, ngram=2)

In [30]:
train_x_spacy_dep_cnt_feature, test_x_spacy_dep_cnt_feature = count_vec(train_spacy_features["dep_txt"].values, test_spacy_features["dep_txt"].values, lowercase=False, ngram=1)

In [31]:
from sarvam.eda.meta_features.text import *

In [ ]:
# add here